In [32]:
%cd ..
from API import *
from itertools import product
from pprint import pprint
from progressbar import ProgressBar
from tqdm.notebook import tqdm
from IPython.display import display

import tensorflow as tf
import numpy as np
import datetime
import threading
import warnings

warnings.filterwarnings('ignore')

e:\REMOTE\YandexDisk\MPEI


In [33]:
in_df_list = []
out_df_list = []

path = 'E:/REMOTE/YandexDisk/MPEI/DIP/part 2/split/network/'

for i in range(10):
    in_df_list.append(pd.read_parquet(path + 'in_stages_thread_{}.parquet'.format(i)))
    out_df_list.append(pd.read_parquet(path + 'out_stages_thread_{}.parquet'.format(i)))

in_df = pd.concat(in_df_list, ignore_index=True)
out_df = pd.concat(out_df_list, ignore_index=True)

In [34]:
main_df = pd.concat([in_df, out_df], axis=1)
main_df = main_df[main_df['isOK'] == 1]
main_df.reset_index(drop=True, inplace=True)

main_df.to_excel(path + 'main_data.xlsx', index=False)

main_df

,p_0,t_0,H_0,reaction,d_hub,n,alpha_1,G_0,isOK,eff_ol,Nu,l_1
0,2.0,300.0,20.0,0.10,0.65,50.0,13.0,20.0,1.0,0.926155,0.381806,0.028634
1,2.0,300.0,20.0,0.10,0.65,50.0,25.0,20.0,1.0,0.863708,0.342843,0.015154
2,2.0,300.0,20.0,0.10,0.65,50.0,29.0,20.0,1.0,0.833342,0.325455,0.013844
3,2.0,300.0,40.0,0.10,0.75,50.0,13.0,20.0,1.0,0.885774,0.724548,0.019232
4,2.0,300.0,40.0,0.15,0.75,50.0,15.0,60.0,1.0,0.920837,2.235250,0.042189
...,...,...,...,...,...,...,...,...,...,...,...,...
7194,8.0,500.0,140.0,0.45,0.80,50.0,15.0,100.0,1.0,0.705086,8.625944,0.019094
7195,8.0,500.0,140.0,0.45,0.80,50.0,15.0,140.0,1.0,0.714437,12.219902,0.026493
7196,8.0,500.0,140.0,0.45,0.80,50.0,17.0,100.0,1.0,0.698161,8.522121,0.016948
7197,8.0,500.0,140.0,0.50,0.80,50.0,23.0,100.0,1.0,0.672228,8.072979,0.013124


In [44]:
train_df = main_df.iloc[::2]
test_df = main_df.iloc[1::2]


,p_0,t_0,H_0,reaction,d_hub,n,alpha_1,G_0,isOK,eff_ol,Nu,l_1
1,2.0,300.0,20.0,0.10,0.65,50.0,25.0,20.0,1.0,0.863708,0.342843,0.015154
3,2.0,300.0,40.0,0.10,0.75,50.0,13.0,20.0,1.0,0.885774,0.724548,0.019232
5,2.0,300.0,40.0,0.25,0.85,50.0,11.0,60.0,1.0,0.953858,2.359463,0.061727
7,2.0,300.0,40.0,0.25,0.95,50.0,23.0,60.0,1.0,0.901287,2.162280,0.028144
9,2.0,300.0,40.0,0.30,0.85,50.0,23.0,100.0,1.0,0.905660,3.591347,0.052204
...,...,...,...,...,...,...,...,...,...,...,...,...
7189,8.0,500.0,140.0,0.15,0.85,100.0,13.0,60.0,1.0,0.905387,7.926666,0.017682
7191,8.0,500.0,140.0,0.25,0.65,100.0,13.0,60.0,1.0,0.850626,7.163371,0.014967
7193,8.0,500.0,140.0,0.35,0.80,100.0,23.0,20.0,1.0,0.893370,2.465836,0.031633
7195,8.0,500.0,140.0,0.45,0.80,50.0,15.0,140.0,1.0,0.714437,12.219902,0.026493


In [93]:
in_train_new_df = pd.concat([train_df['p_0'], 
                      train_df['t_0'],
                      train_df['H_0'], 
                      train_df['reaction'], 
                      train_df['d_hub'],
                      train_df['n'],
                      train_df['alpha_1'],
                      train_df['G_0']], axis=1)
out_train_new_df = pd.concat([train_df['eff_ol'],
                        train_df['Nu'],
                        train_df['l_1']], axis=1)

in_test_new_df = pd.concat([test_df['p_0'], 
                      test_df['t_0'],
                      test_df['H_0'], 
                      test_df['reaction'], 
                      test_df['d_hub'],
                      test_df['n'],
                      test_df['alpha_1'],
                      test_df['G_0']], axis=1)
out_test_new_df = pd.concat([test_df['eff_ol'],
                        test_df['Nu'],
                        test_df['l_1']], axis=1)

in_test_new_df = tf.convert_to_tensor(in_test_new_df)
out_test_new_df = tf.convert_to_tensor(out_test_new_df)
in_train_new_df['alpha_1'].unique()

array([13., 29., 15.,  9., 17., 25., 23., 21., 19., 27., 11.])

In [37]:
# in_ten = tf.convert_to_tensor(in_new_df)
# out_ten = tf.convert_to_tensor(out_new_df)
# in_ten

In [38]:
number_of_input_parameters = in_test_new_df.shape[1]
number_of_output_parameters = out_test_new_df.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Dense(number_of_input_parameters, activation = 'relu', input_dim=number_of_input_parameters),
    tf.keras.layers.Dense(number_of_input_parameters * 2, activation = 'relu'),
    tf.keras.layers.Dense(number_of_output_parameters, activation=None)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics='accuracy'
)
history = model.fit(x=in_train_new_df, y=out_train_new_df, epochs=1000)   

Epoch 1/1000
113/113 [==============================] - 0s 651us/step - loss: 82.2665 - accuracy: 0.7067
Epoch 2/1000
113/113 [==============================] - 0s 683us/step - loss: 12.1309 - accuracy: 0.8367
Epoch 3/1000
113/113 [==============================] - 0s 632us/step - loss: 5.4201 - accuracy: 0.9831
Epoch 4/1000
113/113 [==============================] - 0s 649us/step - loss: 2.6247 - accuracy: 0.9778
Epoch 5/1000
113/113 [==============================] - 0s 637us/step - loss: 1.8439 - accuracy: 0.9706
Epoch 6/1000
113/113 [==============================] - 0s 679us/step - loss: 1.5337 - accuracy: 0.9703
Epoch 7/1000
113/113 [==============================] - 0s 638us/step - loss: 1.3679 - accuracy: 0.9706
Epoch 8/1000
113/113 [==============================] - 0s 634us/step - loss: 1.2497 - accuracy: 0.9719
Epoch 9/1000
113/113 [==============================] - 0s 644us/step - loss: 1.1805 - accuracy: 0.9725
Epoch 10/1000
113/113 [==============================] - 0s 62

In [39]:
test_loss, test_acc = model.evaluate(in_test_new_df, out_test_new_df,verbose=2)
print("acc:", test_acc)

113/113 - 0s - loss: 0.0496 - accuracy: 0.9900 - 121ms/epoch - 1ms/step
acc: 0.9899972081184387


In [40]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [94]:
pre_df = pd.DataFrame({
    "p_0":      [3.245,0.792,4.564, 3.3],       
    "t_0":      [366.165, 234.622, 304.926, 350],
    "H_0":      [34.727, 67.525, 37.074, 33.49],
    "reaction": [0.242, 0.174, 0.228, 0.189],
    "d_hub":    [0.822, 1.097,0.869, 0.889],
    "n":        [50,50,50,50],
    "alpha_1":  [13, 12, 12, 13],
    "G_0":      [187.488, 68.56, 238.559, 123.45]
})

# 0.891 / 
pre_df

,p_0,t_0,H_0,reaction,d_hub,n,alpha_1,G_0
0,3.245,366.165,34.727,0.242,0.822,50,13,187.488
1,0.792,234.622,67.525,0.174,1.097,50,12,68.560
2,4.564,304.926,37.074,0.228,0.869,50,12,238.559
3,3.300,350.000,33.490,0.189,0.889,50,13,123.450


In [99]:
# p_0	t_0	H_0	reaction	d_hub	n	alpha_1	G_0
# [3.245, 366.165, 34.727, 0.242, 0.822, 50, 13, 187.488]
prediction = model.predict(pre_df)

out_pre_df = pd.DataFrame(columns=['eff_ol', 'Nu', 'l_1'])

for row in prediction:
    out_pre_df.loc[len(out_pre_df.index)] = row

out_pre_df

1/1 [==============================] - 0s 13ms/step


,eff_ol,Nu,l_1
0,0.752908,7.380511,0.028112
1,0.799243,4.904841,0.028864
2,-2.150153,9.031466,0.095426
3,0.822357,4.966993,0.031408


In [97]:
out_test_new_df

<tf.Tensor: shape=(3599, 3), dtype=float64, numpy=
array([[ 0.86370757,  0.34284313,  0.01515398],
       [ 0.8857737 ,  0.72454763,  0.01923224],
       [ 0.95385779,  2.35946346,  0.0617267 ],
       ...,
       [ 0.89336963,  2.46583551,  0.03163278],
       [ 0.71443724, 12.21990179,  0.02649293],
       [ 0.67222814,  8.07297904,  0.01312441]])>